# Imports

In [ ]:
# required imports
from firebolt.service.manager import ResourceManager
from firebolt.client import DEFAULT_API_URL
from firebolt.client.auth import UsernamePassword, ServiceAccount

# use debug function for pretty printing
from devtools import debug

# Logging

In [ ]:
# configure logging
import logging

logging.basicConfig(
    format="{asctime} - {name} - {levelname} - {message}", style="{", level="INFO"
)

# uncomment to show every web request
# logging.getLogger("firebolt.client").setLevel("DEBUG")

# uncomment to suppress info messages from core database/engine operations
# logging.getLogger("firebolt.service").setLevel("WARNING")
# logging.getLogger("firebolt.model").setLevel("WARNING")

# uncomment to suppress info messages from running queries
# logging.getLogger("firebolt.async_db").setLevel("WARNING")

# Configure ResourceManager

**Option 1**: create a `.env` file with the following contents (fill in values):
```
FIREBOLT_USER=''
FIREBOLT_PASSWORD=''
FIREBOLT_SERVER=''
FIREBOLT_DEFAULT_REGION=''
```
(or ensure these env vars are already set)

In [ ]:
settings = None

**Option 2**: Specify settings manually using the Settings object:

In [ ]:
# Using username/password authentication
username = ""
password = ""

# Using service account authentication
client_id = ""
client_secret = ""

assert not (username and client_id), "Specify only one set of credentials"
if username and password:
    auth = UsernamePassword(username, password)
else:  # client_id and client_secret
    auth = ServiceAccount(client_id, client_secret)

account_name = ""
server = DEFAULT_API_URL
default_region = "us-east-1"

In [ ]:
from firebolt.common import Settings

settings = Settings(
    auth=auth,
    account_name=account_name,
    server=server,
    default_region=default_region,
)

# Initialize ResourceManager

In [ ]:
rm = ResourceManager(settings=settings)

# see your account_id
debug(rm.account_id)

# see your settings
debug(rm.settings)

Specify a database_name and engine_name below if you want, or run as-is to use the defaults.

In [ ]:
import time

default_name = f"temp_{int(time.time())}"
database_name = default_name  # Change to use you own name for creating a new database
engine_name = default_name  # Change to use you own name for creating a new engine

# Database

### Create database

In [ ]:
database = rm.databases.create(name=database_name, region="us-east-1")
debug(database)

### Get by name

In [ ]:
database = rm.databases.get_by_name(name=database_name)
debug(database)

### Get engines attached to a database

In [ ]:
engines = database.get_attached_engines()

debug(engines)

# Engine

### Create engine

In [ ]:
engine = rm.engines.create(name=engine_name)
debug(engine)

### Get by name

In [ ]:
engine = rm.engines.get_by_name(name=engine_name)
debug(engine)

### Attach to database

In [ ]:
engine.attach_to_database(database=rm.databases.get_by_name(name=database_name))

### Start

In [ ]:
engine = engine.start()
debug(engine)

### Run a query

In [ ]:
with engine.get_connection() as connection:
    with connection.cursor() as cursor:
        result = cursor.execute("select 1")
        debug(result)

### Stop

In [ ]:
stopped_engine = engine.stop()
debug(stopped_engine)

### Get an engine's database

In [ ]:
debug(engine.database)

# Region, Instance Types

### Get default region, and an instance_type
An example of these might be:
 * provider: AWS
 * region: us-east-1
 * instance_type: i3.4xlarge

In [ ]:
debug(rm.regions.default_region)
debug(rm.instance_types.get_by_name(instance_type_name="B2"))